In [1]:
from itertools import product
from random import random, randint, shuffle, seed
import numpy as np
from scipy import sparse
from copy import copy
from tqdm.autonotebook import tqdm
from functools import reduce

/var/folders/7_/2rs9m9mj3fnfnkpplqvc0ph00000gn/T/ipykernel_4963/2987716514.py:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
def make_set_covering_problem(num_points, num_sets, density, prob):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points * 2654435761 + num_sets + density)
    np.random.seed(int(num_points * 435761 + num_sets + density))
    sets = sparse.lil_array((num_sets, num_points), dtype=bool).toarray()
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets - 1), p] = True
    initial_state = np.random.choice([True, False], size=(num_sets,), p=[prob, 1 - prob])
    return sets, initial_state

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [3]:
NUM_SETS = 1000
NUM_POINTS = 1000
DENSITY = 0.3
PROB = 0.02

problem, initial_state = make_set_covering_problem(NUM_POINTS, NUM_SETS, DENSITY, PROB)
print(
    f'Problem shape: {problem.shape}',
    f'Initial state shape: {initial_state.shape}, Taken sets: {np.sum(initial_state)}',
    sep='\n',
)

Problem shape: (1000, 1000)
Initial state shape: (1000,), Taken sets: 10


In [4]:
def check_goal(problem, state):
    return np.all(
        reduce(
            np.logical_or,
            [problem[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(NUM_POINTS)]),
        )
    )

In [5]:
assert check_goal(problem, np.full((NUM_SETS,), True)), "Problem not solvable"

In [6]:
def fitness(problem, state):
    goal = check_goal(problem, state)
    cost = np.sum(state)
    return goal, cost if not goal else -cost

In [7]:
fitness(problem, initial_state)

(False, 10)

In [8]:
def tweak(state):
    new_state = copy(state)
    index = randint(0, NUM_SETS - 1)
    new_state[index] = not new_state[index]
    return new_state

In [11]:
def RHMC(problem, state, fitness, max_it, max_give_up):
    """
    Random-Mutation Hill Climber implementation.

    Args:
        problem: 2-D boolean array;
        state: 1-D boolean array (dim equal to #rows of problem);
        fitness: fitness function to evaluate a state;
        max_it: maximum number of iterations;
        max_give_up: maximum number of evaluations before giving up.
    """
    changes = 0
    evals_giveup = 0
    for evals in tqdm(range(max_it)):
        new_state = tweak(state)
        if fitness(problem, new_state) > fitness(problem, state):
            state = new_state
            changes += 1
            evals_giveup = 0
        else:
            evals_giveup += 1
            if evals_giveup == max_give_up:
                print('Maximum number of evaluations without improvement reached.')
                break
        evals += 1

    changes += 1
    evals += 1
    if evals == max_it:
        print('Maximum number of iterations reached.')
    print(f'Terminated after {changes} changes.', f'Number of evaluations: {evals}.', sep='\n')

    goal, cost = fitness(problem, state)

    print(f'Goal reached? {"Yes" if goal else "No"}', f'State cost: {abs(cost)}', sep='\n')
    return state

In [12]:
_ = RHMC(problem, initial_state, fitness, 10_000, 2_000)

  0%|          | 0/10000 [00:00<?, ?it/s]

Maximum number of evaluations without improvement reached.
Terminated after 14 changes.
Number of evaluations: 2393.
Goal reached? Yes
State cost: 15
